In [ ]:
# 분석 환경 모듈
import numpy as np
import pandas as pd
import psycopg2 as pg
import time
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style

# 데이터 분석 모듈
from CoolProp.CoolProp import PropsSI
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer, PolynomialFeatures
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from scipy.integrate import ode

# 내가 만든 모듈 임포트
from Model import *
from PhysicalProperty import *
from StructuredQuery import *
from Numeric import *

style.use('seaborn-talk')
krfont = {'family':'Times New Roman', 'weight':'bold', 'size':10}
matplotlib.rc('font', **krfont)
matplotlib.rcParams['axes.unicode_minus'] = False

In [ ]:
#############################################################
###################### CHF 계산 과정 ############################
#############################################################

sql = StructuredQuery()
pro = PhysicalProperty()
mod = ModelOSV()

In [ ]:
# CHF 데이터베이스 연결
# SQL 연결 쿼리문
(conn, db_engine) = sql.connect('localhost', 'Research', 'postgres', '5432', '1234')

# 연결 확인
print("PostgreSQL에 Research Database에 연결을 완료하였습니다.", conn)
print(db_engine)

# OSV 데이터베이스 연결
load_chf_query = "SELECT * FROM rawdata_chf_park_tb" # Water CHF Table 10500개
#load_chf_query = "SELECT * FROM rawdata_chf_r12_tb" # R12 CHF Table 10500개
chf_tb = sql.read_sql(load_chf_query, db_engine)

In [ ]:
# Rawdata에서 Physical property 추가 (for water)
chf_tb['pcrit'] = round(chf_tb[['refri']].apply(lambda x: PropsSI(x[0],'pcrit') * 10**-5, axis=1),6) # [bar]
chf_tb['tsat'] = round(chf_tb[['p', 'refri']].apply(lambda x: PropsSI('T', 'P', x[0] * 1e5, 'Q', 0, x[1]), axis=1),6) # [K]
chf_tb['kf'] = round(chf_tb[['p', 'refri']].apply(lambda x: PropsSI('L', 'P', x[0] * 1e5, 'Q', 0, x[1]), axis=1),6) # [W/mK]
chf_tb['kv'] = round(chf_tb[['p', 'refri']].apply(lambda x: PropsSI('L', 'P', x[0] * 1e5, 'Q', 1, x[1]), axis=1),6) # [W/mK]
chf_tb['muf'] = round(chf_tb[['p', 'refri']].apply(lambda x: PropsSI('V', 'P', x[0] * 1e5, 'Q', 0, x[1]), axis=1),12) #  [Pa s]
chf_tb['muv'] = round(chf_tb[['p', 'refri']].apply(lambda x: PropsSI('V', 'P', x[0] * 1e5, 'Q', 1, x[1]), axis=1),12) # [Pa s]
chf_tb['hfo'] = round(chf_tb[['p', 'refri']].apply(lambda x: PropsSI('H', 'P', x[0] * 1e5, 'Q', 0, x[1]), axis=1),6) # [J/kgK]
chf_tb['hgo'] = round(chf_tb[['p', 'refri']].apply(lambda x: PropsSI('H', 'P', x[0] * 1e5, 'Q', 1, x[1]), axis=1),6) #[J/kgK]
chf_tb['lam'] = round(chf_tb['hgo'] - chf_tb['hfo'],6) # [J/kgK]
chf_tb['rhof'] = round(chf_tb[['p', 'refri']].apply(lambda x: PropsSI('D', 'P', x[0] * 1e5, 'Q', 0, x[1]), axis=1),6) #[kg/m3]
chf_tb['rhov'] = round(chf_tb[['p', 'refri']].apply(lambda x: PropsSI('D', 'P', x[0] * 1e5, 'Q', 1, x[1]), axis=1),6) # [kg/m3]
chf_tb['v'] = round(chf_tb['g'] / chf_tb['rhof'],6) # [m/s]
chf_tb['cpf'] = round(chf_tb[['p', 'refri']].apply(lambda x: PropsSI('C', 'P', x[0] * 1e5, 'Q', 0, x[1]), axis=1),6) # [J/kgK]
chf_tb['cpv'] = round(chf_tb[['p', 'refri']].apply(lambda x: PropsSI('C', 'P', x[0] * 1e5, 'Q', 1, x[1]), axis=1),6) # [J/kgK]
chf_tb['sigma'] = round(chf_tb[['p', 'refri']].apply(lambda x: PropsSI('I', 'P', x[0] * 1e5, 'Q', 0, x[1]), axis=1),6) # [N/m]


# Rawdata에서 Radata 항목 추가
#chf_tb['dtin'] = round(chf_tb[['degsubin', 'cpf', 'cpv']].apply(lambda x: x[0]*1e3/x[1] if x[0] > 0 else x[0]*1e3/x[2], axis=1),6)
chf_tb['ti'] = round(chf_tb['tsat'] - chf_tb['dtin'] - 273.15,6) # [K]


# Rawdata에서 Dimensionless number 추가
chf_tb['xi'] = round(chf_tb[['cpf', 'dtin', 'lam']].apply(lambda x: pro.calXi(x[0], x[1], x[2]), axis = 1),6)
chf_tb['xout'] = round(chf_tb[['q', 'doi', 'dio', 'geo', 'hsur', 'g', 'cpf', 'lam', 'dtin', 'lh']].apply(lambda x: pro.calXout(x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9]), axis=1),6)
chf_tb['de'] = round(chf_tb[['doi', 'dio', 'geo', 'hsur', 'dh']].apply(lambda x: pro.calDe(x[0], x[1], x[2], x[3], x[4]), axis=1),6)
chf_tb['pe'] = round(chf_tb[['dh', 'g', 'cpf', 'kf']].apply(lambda x: pro.calPe(x[0], x[1], x[2], x[3]), axis=1),6)
chf_tb['re'] = round(chf_tb[['g', 'dh', 'muf']].apply(lambda x: pro.calRe(x[0], x[1], x[2]), axis=1),6)
chf_tb['we'] = round(chf_tb[['rhof', 'v', 'dh', 'sigma']].apply(lambda x: pro.calWe(x[0], x[1], x[2], x[3]), axis=1),6)
chf_tb['bd'] = round(chf_tb[['rhof', 'rhov', 'dh', 'sigma']].apply(lambda x: pro.calBd(x[0], x[1], x[2], x[3]), axis=1),6)
chf_tb['bo'] = round(chf_tb[['q', 'lam', 'g']].apply(lambda x: pro.calBo(x[0], x[1], x[2]), axis = 1),6)
chf_tb['fr'] = round(chf_tb[['v', 'dh']].apply(lambda x: pro.calFr(x[0], x[1]), axis=1),6)
chf_tb['ca'] = round(chf_tb[['muf', 'v', 'sigma', 'rhof']].apply(lambda x: pro.calCa(x[0], x[1], x[2], x[3]), axis=1),6)
chf_tb['pr'] = round(chf_tb[['cpf', 'muf', 'kf']].apply(lambda x: pro.calPr(x[0], x[1], x[2]), axis=1),6)

print("chf_tb에 Physical property 계산 완료")

In [ ]:
# OSV, CHF 설정 모델 삽입 문구

print('chf correlation을 선택하시오: ')
print('(1) Saha and Zuber  (2) Modified Saha and Zuber  (3) Unal')
print('(4) Levy                         (5) Bowring                                    (6) Jeong')
m_idx = int(input('-----너의 선택은? : '))

In [ ]:
# 결과테이블 새로 생성
res_tb = chf_tb

# 모델별로 XOSV를 계산하기 위한 알고리즘
# def sub_xt_return(p, pcrit, dh, g, lam, m_idx):
if m_idx == 1:
    nm = 'sz'
    for i, row in res_tb.iterrows():
        res_tb.loc[i, 'dt_sz'], res_tb.loc[i, 'x_sz'] = mod.calSahaZuber(res_tb.loc[i, 'q'], res_tb.loc[i, 'rhof'], res_tb.loc[i, 'dh'], res_tb.loc[i, 'g'], 
                                               res_tb.loc[i, 'cpf'], res_tb.loc[i, 'kf'], res_tb.loc[i, 'pe'], res_tb.loc[i, 'lam'])
        res_tb.loc[i,'st_cal'] = res_tb.loc[i,'q']* 10 ** 6 / (res_tb.loc[i,'g'] * res_tb.loc[i,'cpf'] * res_tb.loc[i,'dt_sz'])
    print("Saha and Zuber correlation에 대한 St, Xosv 계산이 모두 끝났습니다.")
elif m_idx == 2:
    nm = 'psz'
    for i, row in res_tb.iterrows():
        res_tb.loc[i, 'dt_psz'], res_tb.loc[i, 'x_psz'] = mod.calParkSahaZuber(res_tb.loc[i, 'q'], res_tb.loc[i, 'rhof'], res_tb.loc[i, 'dh'], res_tb.loc[i, 'g'],
                                               res_tb.loc[i, 'cpf'], res_tb.loc[i, 'kf'], res_tb.loc[i, 'pe'], res_tb.loc[i, 'lam'])
        res_tb.loc[i,'st_cal'] = res_tb.loc[i,'q'] * 10 ** 6 / (res_tb.loc[i,'g'] * res_tb.loc[i,'cpf'] * res_tb.loc[i,'dt_psz'])
    print("Park, Saha and Zuber correlation에 대한 St, Xosv 계산이 모두 끝났습니다.")
elif m_idx == 3:
    nm = 'unal'
    for i, row in res_tb.iterrows():
        res_tb.loc[i, 'dt_unal'], res_tb.loc[i, 'x_unal'] = mod.calUnal(res_tb.loc[i, 'q'], res_tb.loc[i, 'pr'], res_tb.loc[i, 'dh'], res_tb.loc[i, 'v'],
                                            res_tb.loc[i, 'cpf'], res_tb.loc[i, 'kf'], res_tb.loc[i, 're'], res_tb.loc[i, 'refri'], res_tb.loc[i, 'lam'])
        res_tb.loc[i,'st_cal'] = res_tb.loc[i,'q'] * 10 ** 6 / (res_tb.loc[i,'g'] * res_tb.loc[i,'cpf'] * res_tb.loc[i,'dt_unal'])
    print("Unal correlation에 대한 St, Xosv 계산이 모두 끝났습니다.")
elif m_idx == 4:
    nm = 'levy'
    for i, row in res_tb.iterrows():
        res_tb.loc[i, 'dt_levy'], res_tb.loc[i, 'x_levy'] = mod.calLevy(res_tb.loc[i, 'sigma'], res_tb.loc[i, 'dh'], res_tb.loc[i, 'rhof'], res_tb.loc[i, 'muf'],
                                            res_tb.loc[i, 'kf'], res_tb.loc[i, 're'], res_tb.loc[i, 'pr'], res_tb.loc[i, 'cpf'],
                                            res_tb.loc[i, 'g'], res_tb.loc[i, 'q'], res_tb.loc[i, 'lam'], res_tb.loc[i, 'v'])
        res_tb.loc[i,'st_cal'] = res_tb.loc[i,'q']* 10 ** 6 / (res_tb.loc[i,'g'] * res_tb.loc[i,'cpf'] * res_tb.loc[i,'dt_levy'])
    print("Levy Model에 대한 St, Xosv 계산이 모두 끝났습니다.")
elif m_idx == 5:
    nm = 'bowr'
    for i, row in res_tb.iterrows():
        res_tb.loc[i, 'dt_bowr'], res_tb.loc[i, 'x_bowr'] = mod.calBowring(res_tb.loc[i, 'p'], res_tb.loc[i, 'q'], res_tb.loc[i, 'v'], res_tb.loc[i, 'lam'],
                                                  res_tb.loc[i, 'cpf'])
        res_tb.loc[i,'st_cal'] = res_tb.loc[i,'q'] * 10 ** 6 / (res_tb.loc[i,'g'] * res_tb.loc[i,'cpf'] * res_tb.loc[i,'dt_bowr'])
    print("Bowring correlation에 대한 St, Xosv 계산이 모두 끝났습니다.")
else:
    nm = 'js'
    for i, row in res_tb.iterrows():
        res_tb.loc[i, 'dt_js'], res_tb.loc[i, 'x_js'] = mod.calJeong(res_tb.loc[i, 'q'], res_tb.loc[i, 'rhof'], res_tb.loc[i, 'rhov'], res_tb.loc[i, 'dh'], res_tb.loc[i, 'v'],
                                           res_tb.loc[i, 'cpf'], res_tb.loc[i, 'kf'], res_tb.loc[i, 'pe'], res_tb.loc[i, 'lam'],res_tb.loc[i, 'pr'],res_tb.loc[i, 'we'],res_tb.loc[i, 'ca'])
        res_tb.loc[i,'st_cal'] = res_tb.loc[i,'q'] * 10 ** 6 / (res_tb.loc[i,'g'] * res_tb.loc[i,'cpf'] * res_tb.loc[i,'dt_js'])
    print("Jeong and Shim correlation에 대한 St, Xosv 계산이 모두 끝났습니다.")    

In [ ]:
# 데이터를 PostgreSQL로 저장하기
"""sql.write_sql(val_chf_tb, 'val_res_r12_'+nm+'_tb',db_engine)
sql.write_sql(val_chf_err_tb, 'val_res_r12_'+nm+'_err_tb', db_engine)
sql.write_sql(val_chf_xt_tb, 'val_chf_r12_'+nm+'_xt_tb', db_engine)
sql.write_sql(chf_tb, 'pp_chf_r12_tb',db_engine)"""

In [ ]:
# Validation set와  error table 만드는 작업 진행
val_chf_tb = res_tb.loc[0:5,['dh','lh', 'g','q','xi','xe','xout','st_cal']]
val_chf_err_tb = res_tb.loc[0:5,['g','q', 'xi','xe','xout','st_cal']]

# Diverged data의 Xt를 추적하기 위한 테이블
#val_chf_xt_tb = pd.DataFrame()
#val_chf_xt_tb.loc[:, 'data_id'] = ""
#val_chf_xt_tb.loc[:, 'cnt_tr'] = ""

func = lambda x: val_chf_tb.loc[i,'xosv_cal'] * np.log((val_chf_tb.loc[i,'xeq'] -x)/val_chf_tb.loc[i, 'xb']) + np.log((1-val_chf_tb.loc[i, 'xeq']+val_chf_tb.loc[i,'xosv_cal']-val_chf_tb.loc[i,'xosv_cal']*x)/(1-val_chf_tb.loc[i, 'xb']+val_chf_tb.loc[i, 'xosv_cal']))


for i, row in val_chf_tb.iterrows():
    # 앞서 계산한 OSV 상관식으로 계산한 st이 삽입된 xosv_cal 계산
    val_chf_tb.loc[i,'xosv_cal'] = round(-(val_chf_tb.loc[i,'q'] * 10**6)/ (val_chf_tb.loc[i,'st_cal'] * val_chf_tb.loc[i,'g'] * res_tb.loc[i,'lam']),6) # XOSV 계산값 (based on qCHF)
    val_chf_tb.loc[i,'xb'] = round(max(res_tb.loc[i,'xi'], val_chf_tb.loc[i, 'xosv_cal']), 6) # (Xb, XOSV) 계산값 (based on qCHF)
    
    #xt_cal 초기값
    val_chf_tb.loc[i,'xeq'] = round(val_chf_tb.loc[i, 'xi']+ (4*val_chf_tb.loc[i,'q']*10**6*val_chf_tb.loc[i,'lh'])/(res_tb.loc[i,'lam']*val_chf_tb.loc[i,'g']*val_chf_tb.loc[i,'dh']),12)
    tmp = val_chf_tb.loc[i,'xosv_cal'] * np.log(val_chf_tb.loc[i,'xeq']/val_chf_tb.loc[i, 'xb']) + np.log((1-val_chf_tb.loc[i, 'xeq']+val_chf_tb.loc[i,'xosv_cal'])/(1-val_chf_tb.loc[i, 'xb']+val_chf_tb.loc[i, 'xosv_cal']))

    conv_tag = 0

    # xt=0에서 Fxt가 존재하는가? 
    if np.isinf(tmp) == 1 or np.isnan(tmp) == 1:
        print('f(Xt) : {}'.format(round(tmp,6)))
        val_chf_tb.loc[i, 'xt_cal'] = round(val_chf_tb.loc[i, 'xeq'],6)
    else:
        if val_chf_tb.loc[i, 'xeq'] >0:
            val_chf_tb.loc[i, 'xt_cal'] = round(val_chf_tb.loc[i, 'xeq'],6)
        else:
            print('f(Xt) : {}'.format(round(tmp,6)))
            funky = lambda x: val_chf_tb.loc[i,'xosv_cal']*np.log((val_chf_tb.loc[i,'xeq']-x)/val_chf_tb.loc[i,'xb'])
            line = lambda x: -np.log((1-val_chf_tb.loc[i,'xeq']+val_chf_tb.loc[i,'xosv_cal']-val_chf_tb.loc[i,'xosv_cal']*x)/(1-val_chf_tb.loc[i,'xb']+val_chf_tb.loc[i,'xosv_cal']))
            result = findIntersection(funky, line, 0)
            val_chf_tb.loc[i, 'xt_cal'] = result
            conv_tag = 1
            #val_chf_tb.loc[i,'xt_cal'] = 0


    cnt = 0 # Xt 계산 알고리즘 반복회수 최대값
    cnt_nan = 0
    tmp_list = []

    while 1:
        try:
            if conv_tag == 1: # 초기화 단계에서 수렴했으면 CHF 계산으로 넘어감
                val_chf_tb.loc[i,'Fxt'] = 0
                val_chf_err_tb.loc[i, 'converged'] = 'Analytical, Converged'
                continue
            else:
                pass
            cnt += 1 # 시작 계산횟수 1회
            val_chf_tb.loc[i, 'Fxt'] = val_chf_tb.loc[i,'xosv_cal'] * np.log((val_chf_tb.loc[i,'xeq'] -val_chf_tb.loc[i, 'xt_cal'])/val_chf_tb.loc[i, 'xb']) + np.log((1-val_chf_tb.loc[i, 'xeq']+val_chf_tb.loc[i,'xosv_cal']-val_chf_tb.loc[i,'xosv_cal']*val_chf_tb.loc[i, 'xt_cal'])/(1-val_chf_tb.loc[i, 'xb']+val_chf_tb.loc[i, 'xosv_cal']))
            print("Fxt is {}".format(round(val_chf_tb.loc[i, 'Fxt'],6)))
        except ZeroDivisionError as e1:
            print("Error is occurs e1 : {}".format(e1))
            #pass
        except ValueError as e2:
            print("Error is occurs e2 : {}".format(e2))
            #pass
        except KeyError as e3:
            print("Error is occurs e3 : {}".format(e3))
            #pass         
        finally:
            print(i, round(val_chf_tb.loc[i,'xeq'],6), round(val_chf_tb.loc[i, 'xt_cal'], 6), round(val_chf_tb.loc[i,'Fxt'],6))

            # 종료조건 1 : 반복횟수 100회 이상
            if cnt > 5000:
                val_chf_err_tb.loc[i, 'converged'] = 'Diverged'
                break

            # 종료조건 2: nan이 5번 이상이면 종료
            if cnt_nan > 5:
                print('해가 없으므로, 강제로 수렴시킵니다.')
                val_chf_tb.loc[i, 'Fxt'] = round(val_chf_tb.loc[i, 'Fxt_old'],6)
                val_chf_err_tb.loc[i, 'converged'] = 'Xt=Xeq, Converged'
                break

            # 종료조건 3 : xi, Fxt 값에 따른 수렴 여부 파악
            if val_chf_tb.loc[i, 'xi'] > 0: # inlet quality > 0이면 그래프 개형은 왼쪽으로 발산
                if np.abs(val_chf_tb.loc[i, 'Fxt']) < 0.01:
                    val_chf_err_tb.loc[i, 'converged'] = 'Numerical, Converged'
                    break
                else:
                    if val_chf_tb.loc[i, 'Fxt'] < 0:
                        cnt += 1
                        if np.isnan(val_chf_tb.loc[i, 'Fxt']) == 1:
                            val_chf_tb.loc[i,'Fxt_old'] = 0
                            cnt_nan += 1
                            val_chf_tb.loc[i, 'xt_cal'] -=5e-4
                            #print('cnt_nan + 1')
                            continue
                        #val_chf_tb.loc[i,'Fxt_old'] = round(val_chf_tb.loc[i, 'Fxt'],6)
                        
                        # Bisection Method로 값 찾기
                        print('check1')
                        val_chf_tb.loc[i, 'xt_cal'] = round(bisect(func, val_chf_tb.loc[i, 'xt_cal']+5e-4, val_chf_tb.loc[i, 'xt_cal'])[0],6)
                        val_chf_err_tb.loc[i, 'converged'] = 'Bisection, Converged'
                        break
                        #val_chf_tb.loc[i, 'xt_cal'] +=1e-4
                        
                        #continue
                    else:
                        cnt += 1
                        if np.isnan(val_chf_tb.loc[i, 'Fxt']) == 1:
                            val_chf_tb.loc[i,'Fxt_old'] = 0
                            cnt_nan += 1
                            val_chf_tb.loc[i, 'xt_cal'] -=5e-4
                            #print('cnt_nan + 1')
                            continue
                        val_chf_tb.loc[i,'Fxt_old'] = round(val_chf_tb.loc[i, 'Fxt'],6)
                        val_chf_tb.loc[i, 'xt_cal'] -=5e-4
                        #print('check2')
                        continue
            elif val_chf_tb.loc[i, 'xi'] < 0: # inlet quality < 0이면 그래프 개형은 오른쪽으로 발산
                if np.abs(val_chf_tb.loc[i, 'Fxt']) < 0.01:
                    val_chf_err_tb.loc[i, 'converged'] = 'Numerical, Converged'
                    break
                else:
                    if val_chf_tb.loc[i, 'Fxt'] < 0:
                        cnt += 1
                        if np.isnan(val_chf_tb.loc[i, 'Fxt']) == 1:
                            val_chf_tb.loc[i,'Fxt_old'] = 0
                            cnt_nan += 1
                            val_chf_tb.loc[i, 'xt_cal'] +=5e-4
                            #print('cnt_nan + 1')
                            continue
                        #val_chf_tb.loc[i,'Fxt_old'] = round(val_chf_tb.loc[i, 'Fxt'],6)

                        # Bisection Method로 값 찾기
                        val_chf_tb.loc[i, 'xt_cal'] = round(bisect(func, val_chf_tb.loc[i, 'xt_cal']-5e-4, val_chf_tb.loc[i, 'xt_cal'])[0],6)
                        val_chf_err_tb.loc[i, 'converged'] = 'Bisection, Converged'
                        break
                        #val_chf_tb.loc[i, 'xt_cal'] -=5e-4 # Xt_cal 갱신
                        #print('check3')
                        #continue
                    else:
                        cnt += 1
                        if np.isnan(val_chf_tb.loc[i, 'Fxt']) == 1:
                            val_chf_tb.loc[i,'Fxt_old'] = 0
                            cnt_nan += 1
                            val_chf_tb.loc[i, 'xt_cal'] +=5e-4
                            #print('cnt_nan + 1')
                            continue
                        val_chf_tb.loc[i,'Fxt_old'] = round(val_chf_tb.loc[i, 'Fxt'],6)
                        val_chf_tb.loc[i, 'xt_cal'] +=5e-4 # Bisection Method
                        #print('check4')
                        continue
            else:
                val_chf_tb.loc[i, 'Fxt'] = 0
                val_chf_tb.loc[i, 'xt_cal'] = val_chf_tb.loc[i, 'xeq']
                val_chf_err_tb.loc[i, 'converged'] = 'Xi==0, Converged'
                break
    
    cnt_nan = 0 # nan값을 삭제하고 초기화
    
    # CHF 계산 (Park)
    val_chf_tb.loc[i,'alpha_park'] = round(0.71+4.6*res_tb.loc[i,'p']/res_tb.loc[i,'pcrit']-5.33*res_tb.loc[i,'p']/res_tb.loc[i,'pcrit']**2,6)
    val_chf_tb.loc[i,'gamma_park'] = round(0.1-0.58*res_tb.loc[i,'p']/res_tb.loc[i,'pcrit']**2-1.54*res_tb.loc[i,'p']/res_tb.loc[i,'pcrit']**3,6)
    val_chf_tb.loc[i,'k1_park'] = round(-0.343+0.22626*np.log(val_chf_tb.loc[i,'g'])-0.01409*np.log(val_chf_tb.loc[i,'g'])**2,6)
    val_chf_tb.loc[i,'k2_park'] = 0.545
    val_chf_tb.loc[i,'k3_park'] = 2.6404-6.5*val_chf_tb.loc[i,'k1_park']+6.1565*val_chf_tb.loc[i,'k1_park']**2
    val_chf_tb.loc[i,'fxt_park'] = round(np.sqrt(val_chf_tb.loc[i,'xt_cal']*(1+val_chf_tb.loc[i,'xt_cal']**2)**3),6)
    val_chf_tb.loc[i,'q_cal_park'] = round(val_chf_tb.loc[i,'alpha_park']/(val_chf_tb.loc[i,'dh'] ** val_chf_tb.loc[i,'k1_park']) * np.exp (-val_chf_tb.loc[i,'gamma_park']*((val_chf_tb.loc[i,'g']**0.55) * np.sqrt(val_chf_tb.loc[i,'xt_cal'] * (1+(val_chf_tb.loc[i,'xt_cal']**2))**3)**val_chf_tb.loc[i,'k2_park'])),6) # Park

    # CHF 계산 (Deng)
    val_chf_tb.loc[i,'alpha_deng'] = round(1.669-6.544*(res_tb.loc[i,'p']/res_tb.loc[i,'pcrit']-0.448)**2,6)
    val_chf_tb.loc[i,'gamma_deng'] = round(0.06523 + (0.1045/(np.sqrt(2*np.pi*(res_tb.loc[i,'p']/res_tb.loc[i,'pcrit'])**2))) * np.exp(-5.413*((np.log(res_tb.loc[i,'p']/res_tb.loc[i,'pcrit'])+0.4537)**2/(np.log(res_tb.loc[i,'p']/res_tb.loc[i,'pcrit'])**2))),6)
    val_chf_tb.loc[i,'fxt_deng'] = round(np.sqrt(res_tb.loc[i,'g']*val_chf_tb.loc[i,'xt_cal']*((1+val_chf_tb.loc[i,'xt_cal']**2)**3)),6)
    val_chf_tb.loc[i, 'q_cal_deng'] = round((val_chf_tb.loc[i, 'alpha_deng']/np.sqrt(res_tb.loc[i,'dh']))*np.exp(-val_chf_tb.loc[i,'gamma_deng']*val_chf_tb.loc[i,'fxt_deng']),6)

    val_chf_tb.loc[i, 'q_err_park'] = round((val_chf_tb.loc[i,'q_cal_park']/res_tb.loc[i,'q']-1),6)
    val_chf_tb.loc[i, 'q_err_deng'] = round((val_chf_tb.loc[i,'q_cal_deng']/res_tb.loc[i,'q']-1),6)

In [ ]:
sql.write_sql(val_chf_tb, 'test_val_js_tb',db_engine)
sql.write_sql(val_chf_err_tb, 'test_err_js_tb',db_engine)

In [ ]:
sql.write_sql(res_tb, 'text_res_tb', db_engine)